In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

box_DIR = "../data/box/"
img_DIR_T = "../data/images/teams/"
fig_DIR = "../figs/teams/"

In [ ]:
dfa = []
for year in range(2019,2024):
    df1 = pd.read_parquet(box_DIR + f"NBA_Box_T_Base_{year}.parquet")
    df1.insert(0,"GAME_ID",df1.pop("GAME_ID"))
    df4 = df1.groupby("GAME_ID")
    df4_1 = df4.nth(0)
    df4_2 = df4.nth(1)
    df4_1.columns = ["GAME_ID"] + [s + "1" for s in df4_1.columns if s != "GAME_ID"]
    df4_2.columns = ["GAME_ID"] + [s + "2" for s in df4_2.columns if s != "GAME_ID"]
    df4_3 = pd.merge(df4_1, df4_2, on="GAME_ID")
    df4_4 = df4.nth(1)
    df4_5 = df4.nth(0)
    df4_4.columns = ["GAME_ID"] + [s + "1" for s in df4_4.columns if s != "GAME_ID"]
    df4_5.columns = ["GAME_ID"] + [s + "2" for s in df4_5.columns if s != "GAME_ID"]
    df4_6 = pd.merge(df4_4, df4_5, on="GAME_ID")
    df5 = pd.concat([df4_3,df4_6])
    df5["FT_Diff"] = df5["FTA1"] - df5["FTA2"]
    df6 = pd.DataFrame(df5.groupby("TEAM_NAME1")["FT_Diff"].sum()).reset_index()
    df6.columns = ["Team","FT_Diff"]
    df6["Season"] = year + 1
    dfa.append(df6)


In [ ]:
df7 = pd.concat(dfa)
df7 = df7.sort_values("FT_Diff",ascending=False).reset_index(drop=True).reset_index()
df7["index"] += 1  
df7["T"]= df7["Team"]
df7.insert(1,"T",df7.pop("T"))
df7.insert(3,"Season",df7.pop("Season"))
df8 = df7.head(10)

In [ ]:
t = (
    GT(df8)
    .tab_header(
        title=md("Highest FT Differential, Past 5 Seasons"),
        # subtitle="Daily measurements in New York City (May 1-10, 1973)"
    )
    .tab_source_note(source_note="@sradjoker | source: nba.com/stats via nba_api")
    .cols_label(
        index = "#",
        T = "",
        FT_Diff = "FT Diff",
    )
    .fmt_image(
        columns="T",
        path = img_DIR_T,
        file_pattern="{}.png"
    )
    .cols_align(align="center")
    .cols_align(align="left", columns="Team")
    .tab_options(
        heading_title_font_size="150%",
        heading_subtitle_font_size="110%",
        # heading_title_font_weight='bold',
        table_background_color="floralwhite",
        column_labels_font_size="105%",
        column_labels_font_weight='bold',
        row_group_font_weight='bold',
        row_group_background_color="#E5E1D8",
        table_font_size=10,
        table_font_names="Consolas", 
        data_row_padding = "3px",
        table_margin_left = 1,
        table_margin_right = 1,
    )
)
t.save(fig_DIR + "ft_diff.png",scale=3,expand=2,web_driver="edge")
t